Importando bibliotecas

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import chi2_contingency, spearmanr
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [11]:
df = pd.read_csv("D:\\github\\SIGAMA-data-analysis\\data\\dataset_normalizado_anonimizado.csv")

Chi-Square Test

In [12]:
df.columns = df.columns.str.strip().str.lower()

# Criar variável binária de aprovação
df["approved"] = (df["status"] == "APROVADO").astype(int)

In [13]:
contingency = pd.crosstab(df["tipo_de_conta"], df["approved"])

chi2, p, dof, expected = chi2_contingency(contingency)

print("Chi-square:", chi2)
print("p-value:", p)

Chi-square: 9.123288150594922
p-value: 0.0025237593125928787


Spearman Correlation

In [ ]:
df = df.sort_values(["cpf_hash", "ano", "mes_num", "dia"])

df["attempt_number"] = df.groupby("cpf_hash").cumcount() + 1

df.head()

In [14]:
corr_day, p_day = spearmanr(df["dia"], df["approved"])
corr_attempt, p_attempt = spearmanr(df["attempt_number"], df["approved"])

print("Spearman Day vs Approval:", corr_day, p_day)
print("Spearman Attempt vs Approval:", corr_attempt, p_attempt)

KeyError: 'attempt_number'

Logistic Regression

In [ ]:
X = df[["tipo_de_conta", "dia", "mes_num", "attempt_number"]]
y = df["approved"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

print(classification_report(y_test, model.predict(X_test)))

Survival Analysis

In [ ]:
from lifelines import KaplanMeierFitter

kmf = KaplanMeierFitter()

durations = attempts["attempts_until_approval"]
event_observed = attempts["attempts_until_approval"] < \
                 attempts.groupby("cpf_hash")["attempts_until_approval"].transform("max")

kmf.fit(durations, event_observed=event_observed)

kmf.plot()
plt.title("Survival Curve - Attempts Until Approval")
plt.show()